In [13]:
# -----------------------------------------------------------------------------
# قدم ۰: نصب و ایمپورت کتابخانه‌های لازم
# -----------------------------------------------------------------------------
!pip install transformers[torch] datasets evaluate accelerate -U -q

import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import warnings
warnings.filterwarnings("ignore") # نادیده گرفتن هشدارهای غیرضروری

print("✅ قدم ۰: نصب و ایمپورت کتابخانه‌ها با موفقیت انجام شد.")

# -----------------------------------------------------------------------------
# قدم ۱: ایجاد دیتاست به صورت دستی
# -----------------------------------------------------------------------------
data = {
    'train': [
        ("این غذا خیلی خوشمزه بود، واقعا لذت بردم!", 1),
        ("پیک خیلی دیر رسید و غذا کاملا سرد شده بود.", 0),
        ("کیفیت مواد اولیه عالی بود، حتما دوباره سفارش میدم.", 1),
        ("متاسفانه سفارشم اشتباه ارسال شده بود.", 0),
        ("بسته‌بندی خیلی خوب و بهداشتی بود.", 1),
        ("قیمت‌ها نسبت به حجم غذا خیلی بالاست.", 0),
        ("طعم پیتزا معمولی بود، چیز خاصی نداشت.", 0),
        ("از پشتیبانی و پیگیری‌شون خیلی راضی بودم.", 1),
        ("سالاد تازه و خوشمزه بود.", 1),
        ("این رستوران رو به هیچکس پیشنهاد نمیکنم.", 0),
        ("غذا شور بود و قابل خوردن نبود.", 0),
        ("سس مخصوص کنار غذا فوق‌العاده بود!", 1),
    ],
    'validation': [
        ("تجربه خوبی بود، از همه چیز راضی بودم.", 1),
        ("نون ساندویچ بیات بود.", 0),
        ("حجم غذا خیلی کم بود، اصلا سیر نشدم.", 0),
        ("سرعت ارسال سفارش عالی بود.", 1),
    ],
    'test': [
        ("یک تجربه افتضاح! پولم رو دور ریختم.", 0),
        ("همه چیز بی‌نقص بود، ممنون از شما.", 1),
        ("طعم غذا بد نبود ولی خیلی چرب بود.", 0),
        ("ارسال سریع و برخورد خوب پیک.", 1),
    ]
}

df_train = pd.DataFrame(data['train'], columns=['comment', 'label_id'])
df_validation = pd.DataFrame(data['validation'], columns=['comment', 'label_id'])
df_test = pd.DataFrame(data['test'], columns=['comment', 'label_id'])

train_dataset = Dataset.from_pandas(df_train)
validation_dataset = Dataset.from_pandas(df_validation)
test_dataset = Dataset.from_pandas(df_test)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

print("✅ قدم ۱: دیتاست با موفقیت به صورت دستی ایجاد شد.")
print("   ساختار دیتاست:", dataset)

# -----------------------------------------------------------------------------
# قدم ۲: توکنایز کردن دیتاست (به روش صحیح)
# -----------------------------------------------------------------------------
model_name = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    # پدینگ در این مرحله انجام نمی‌شود. به DataCollator سپرده می‌شود.
    return tokenizer(examples["comment"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["comment"])
tokenized_datasets = tokenized_datasets.rename_column("label_id", "labels")
tokenized_datasets.set_format("torch")

print("\n✅ قدم ۲: دیتاست با موفقیت توکنایز شد.")
print("   نمونه‌ای از داده توکنایز شده:", tokenized_datasets['train'][0])

# -----------------------------------------------------------------------------
# قدم ۳: Fine-tuning مدل
# -----------------------------------------------------------------------------
# لود کردن مدل پایه
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# لود کردن متریک ارزیابی
metric = evaluate.load("accuracy")

# تعریف تابع محاسبه متریک
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# تعریف آرگومان‌های آموزش (سازگار با نسخه‌های قدیمی‌تر)
training_args = TrainingArguments(
    output_dir="./sentiment_model_results",
    num_train_epochs=3,
    per_device_train_batch_size=4, # کاهش بچ سایز برای جلوگیری از خطای حافظه GPU
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    do_eval=True,
    eval_steps=3,  # هر 3 استپ (یک epoch در دیتاست ما) ارزیابی کن
    learning_rate=2e-5, # یک learning rate مناسب برای fine-tuning
    weight_decay=0.01,
)

# ساخت DataCollator برای پدینگ داینامیک
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print("\n✅ قدم ۳: تمام مقدمات برای آموزش فراهم شد. شروع فرآیند Fine-tuning...")

# شروع آموزش
trainer.train()

print("\n🎉 فرآیند آموزش با موفقیت به پایان رسید!")


✅ قدم ۰: نصب و ایمپورت کتابخانه‌ها با موفقیت انجام شد.
✅ قدم ۱: دیتاست با موفقیت به صورت دستی ایجاد شد.
   ساختار دیتاست: DatasetDict({
    train: Dataset({
        features: ['comment', 'label_id'],
        num_rows: 12
    })
    validation: Dataset({
        features: ['comment', 'label_id'],
        num_rows: 4
    })
    test: Dataset({
        features: ['comment', 'label_id'],
        num_rows: 4
    })
})


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]


✅ قدم ۲: دیتاست با موفقیت توکنایز شد.
   نمونه‌ای از داده توکنایز شده: {'labels': tensor(1), 'input_ids': tensor([    2,  2802,  4658,  3805,  8477,  2834,  1348,  5668,  5891, 24078,
         1001,     4]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


...

BertForSequenceClassification LOAD REPORT from: HooshvareLab/bert-fa-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok i


✅ قدم ۳: تمام مقدمات برای آموزش فراهم شد. شروع فرآیند Fine-tuning...


Step,Training Loss


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


🎉 فرآیند آموزش با موفقیت به پایان رسید!


In [15]:
from transformers import pipeline

# تعریف مسیر مدل نهایی
final_model_path = "./my_final_model"

# ذخیره کردن مدل (اگر قبلا انجام نشده)
# trainer.save_model(final_model_path)
# tokenizer.save_pretrained(final_model_path)

# ساخت pipeline با مشخص کردن سخت‌افزار CPU
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=final_model_path,
    tokenizer=final_model_path,
    device=-1  # <<<< تغییر کلیدی: مدل را روی CPU اجرا کن
)

print("✅ Pipeline با موفقیت روی CPU اجرا شد.")

# تست مدل
test_comment_1 = "این بهترین پیتزایی بود که در تمام عمرم خوردم!"
test_comment_2 = "غذا یخ کرده بود و پیک هم نیم ساعت تاخیر داشت. افتضاح!"

result_1 = sentiment_pipeline(test_comment_1)
result_2 = sentiment_pipeline(test_comment_2)

print(f"جمله: '{test_comment_1}' -> نتیجه: {result_1}")
print(f"جمله: '{test_comment_2}' -> نتیجه: {result_2}")

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

✅ Pipeline با موفقیت روی CPU اجرا شد.
جمله: 'این بهترین پیتزایی بود که در تمام عمرم خوردم!' -> نتیجه: [{'label': 'LABEL_1', 'score': 0.5089386105537415}]
جمله: 'غذا یخ کرده بود و پیک هم نیم ساعت تاخیر داشت. افتضاح!' -> نتیجه: [{'label': 'LABEL_0', 'score': 0.5379354953765869}]
